## STANDARTISATION OF COUNTRY NAMES TO ISO3 CODES

In [1]:
import pycountry
from pathlib import Path
import pandas as pd


In [2]:
PROJECT_ROOT = Path.cwd().parents[0]  # adjust if needed
PROCESSED = PROJECT_ROOT / "data" / "processed"

In [3]:
# Target
cpj = pd.read_csv(PROCESSED / "target" / "target_journalist_killings.csv")

# Military
sipri = pd.read_csv(PROCESSED / "military" / "sipri_trade_register.csv")
mip = pd.read_csv(PROCESSED / "military" / "mip_us_interventions.csv")
ucdp = pd.read_csv(PROCESSED / "military" / "ucdp_esd_ty.csv")

# Colonial
coldat = pd.read_csv(PROCESSED / "colonial" / "coldat_colonial_ties.csv")

# Economic
aiddata = pd.read_csv(PROCESSED / "economic" / "aiddata_bilateral_flows.csv")
debt = pd.read_csv(PROCESSED / "economic" / "worldbank_bilateral_debt.csv")

# Control variable
oda = pd.read_csv(PROCESSED / "control-variable" / "oda_received_by_country.csv")

In [ ]:
print("=== CPJ ===")
print(cpj.columns.tolist())
print(cpj.iloc[:, 0].unique()[:10])  # first column, likely country

print("\n=== SIPRI ===")
print(sipri.columns.tolist())
print("Suppliers:", sipri.iloc[:, 0].unique()[:10])
print("Recipients:", sipri.iloc[:, 1].unique()[:10])

print("\n=== MIP ===")
print(mip.columns.tolist())
print(mip.iloc[:, 0].unique()[:10])

print("\n=== UCDP ===")
print(ucdp.columns.tolist())
print(ucdp.iloc[:, 0].unique()[:10])

print("\n=== COLDAT ===")
print(coldat.columns.tolist())
print("Colonies:", coldat.iloc[:, 0].unique()[:10])
print("Colonizers:", coldat.iloc[:, 1].unique()[:10])

print("\n=== AidData ===")
print(aiddata.columns.tolist())
print("Donors:", aiddata.iloc[:, 0].unique()[:10])
print("Recipients:", aiddata.iloc[:, 1].unique()[:10])

print("\n=== ODA ===")
print(oda.columns.tolist())
print(oda.iloc[:, 0].unique()[:10])

print("\n=== Debt ===")
print(debt.columns.tolist())
print("Debtors:", debt.iloc[:, 0].unique()[:10])
print("Creditors:", debt.iloc[:, 2].unique()[:10])

In [6]:
%pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 4.7 MB/s  0:00:01 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Helper function to get ISO3 code
def get_iso3(country_name):
    """Try to match country name to ISO3 code"""
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

# Test it
test_names = ['Afghanistan', 'Germany, Fed. Rep. of', 'CUB', 'Government of France']
for name in test_names:
    print(f"{name} -> {get_iso3(name)}")

Afghanistan → AFG
Germany, Fed. Rep. of → None
CUB → CUB
Government of France → None


In [6]:
def clean_country_name(name):
    """Clean country name before pycountry lookup"""
    if pd.isna(name):
        return None
    
    name = str(name).strip()
    
    # Strip "Government of" prefix (UCDP)
    if name.startswith("Government of "):
        name = name.replace("Government of ", "")
    
    # Manual mapping for known problem cases
    manual_map = {
        # Debt dataset variants
        "Germany, Fed. Rep. of": "Germany",
        "Korea, Republic of": "South Korea",
        "Russian Federation": "Russia",
        # COLDAT lowercase
        "britain": "United Kingdom",
        "belgium": "Belgium",
        "france": "France",
        "germany": "Germany",
        "netherlands": "Netherlands",
        "portugal": "Portugal",
        "spain": "Spain",
        "italy": "Italy",
        # Common variants
        "Cote d'Ivoire": "Côte d'Ivoire",
        "Congo - Kinshasa": "Democratic Republic of the Congo",
        "United States of America": "United States",
    }
    
    if name in manual_map:
        name = manual_map[name]
    
    return name


def get_iso3(country_name):
    """Get ISO3 code for a country name"""
    cleaned = clean_country_name(country_name)
    if cleaned is None:
        return None
    try:
        return pycountry.countries.lookup(cleaned).alpha_3
    except LookupError:
        return None


# Test again
test_names = ['Afghanistan', 'Germany, Fed. Rep. of', 'CUB', 'Government of France', 'britain', "Cote d'Ivoire"]
for name in test_names:
    print(f"{name} -> {get_iso3(name)}")

Afghanistan -> AFG
Germany, Fed. Rep. of -> DEU
CUB -> CUB
Government of France -> FRA
britain -> GBR
Cote d'Ivoire -> CIV


In [8]:
# Collect all unique country names from all datasets
all_country_names = set()

# CPJ
all_country_names.update(cpj['Country'].unique())

# SIPRI
all_country_names.update(sipri['supplier'].unique())
all_country_names.update(sipri['recipient'].unique())

# MIP
all_country_names.update(mip['target_country'].unique())

# UCDP
all_country_names.update(ucdp['supporter'].unique())
all_country_names.update(ucdp['target_location'].unique())

# COLDAT
all_country_names.update(coldat['colony'].unique())
all_country_names.update(coldat['colonizer'].unique())

# AidData
all_country_names.update(aiddata['donor'].unique())
all_country_names.update(aiddata['recipient'].unique())

# ODA
all_country_names.update(oda['country'].unique())

# Debt
all_country_names.update(debt['debtor'].unique())
all_country_names.update(debt['creditor'].unique())

print(f"Total unique country names: {len(all_country_names)}")

# Find which ones fail
failed = []
for name in sorted([n for n in all_country_names if pd.notna(n)]):
    if get_iso3(name) is None:
        failed.append(name)

print(f"\nFailed to match: {len(failed)}")
print("\nUnmatched names:")
for name in failed:
    print(f"  - {name}")

Total unique country names: 990

Failed to match: 597

Unmatched names:
  - ACCU
  - ADF
  - AFG, IRQ, SYR, TUR, SOM, YEM, DJI, LBY, CUB, Central Africa, EGY, JOR
  - ALiR
  - AMB
  - ANVC
  - APCLS
  - APRDC
  - AQAP
  - AQIM
  - ARS/UIC
  - ASL
  - ATNMC
  - ATTF
  - Abdullah Azzam Brigades
  - Achik Special Dragon Party
  - Afghanistan - United Kingdom, United States of America
  - Afghanistan: Government
  - Afghanistan: Islamic State
  - Africa Eastern and Southern
  - Africa Western and Central
  - Africa, North of Sahara, Regional Programs
  - Africa, Regional Programs Multi-Country
  - Africa, Regional Programs, Regional Programs
  - Africa, South of Sahara Multi-Country
  - Africa, South of Sahara, Regional Programs
  - Africa, South of Sahara, Regional Programs Multi-Country
  - African Capacity Building Foundation (ACBF)
  - African Development Bank (AFDB)
  - African Development Fund (AFDF)
  - African Union**
  - Agwelek Forces
  - Al-Shabaab
  - Algeria: Government
  - Al

In [ ]:
manual_map = {
    # === DEBT DATASET VARIANTS ===
    "Germany, Fed. Rep. of": "Germany",
    "Korea, Republic of": "South Korea",
    "Russian Federation": "Russia",
    
    # === COLDAT LOWERCASE ===
    "britain": "United Kingdom",
    "belgium": "Belgium",
    "france": "France",
    "germany": "Germany",
    "netherlands": "Netherlands",
    "portugal": "Portugal",
    "spain": "Spain",
    "italy": "Italy",
    
    # === CONGO VARIANTS ===
    "Congo - Kinshasa": "Democratic Republic of the Congo",
    "Congo, Dem. Rep.": "Democratic Republic of the Congo",
    "Congo, Democratic Republic of": "Democratic Republic of the Congo",
    "DR Congo": "Democratic Republic of the Congo",
    "DRC": "Democratic Republic of the Congo",
    "Congo - Brazzaville": "Republic of the Congo",
    "Congo, Rep.": "Republic of the Congo",
    "Congo, Republic of": "Republic of the Congo",
    
    # === RUSSIA VARIANTS ===
    "Russia": "Russian Federation",
    "Soviet Union": "Russian Federation",
    
    # === KOREA VARIANTS ===
    "Korea": "South Korea",
    "Korea, Rep.": "South Korea",
    "Korea, Dem. People's Rep.": "North Korea",
    "Korea, Democratic Republic of": "North Korea",
    
    # === COMMON VARIANTS ===
    "United States of America": "United States",
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Cote d`Ivoire": "Côte d'Ivoire",
    "Ivory Coast": "Côte d'Ivoire",
    "Turkey": "Türkiye",
    "Turkiye": "Türkiye",
    "Myanmar (Burma)": "Myanmar",
    "East Timor": "Timor-Leste",
    "Brunei": "Brunei Darussalam",
    "Laos": "Lao People's Democratic Republic",
    "Lao PDR": "Lao People's Democratic Republic",
    "Iran, Islamic Rep.": "Iran",
    "Egypt, Arab Rep.": "Egypt",
    "Yemen, Rep.": "Yemen",
    "Venezuela, RB": "Venezuela",
    "Bahamas, The": "Bahamas",
    "Gambia, The": "Gambia",
    "Cape Verde": "Cabo Verde",
    "Swaziland": "Eswatini",
    "Macedonia, FYR": "North Macedonia",
    "Micronesia, Fed. Sts.": "Micronesia",
    "Bosnia-Herzegovina": "Bosnia and Herzegovina",
    "Bosnia": "Bosnia and Herzegovina",
    "Kosovo": "Kosovo",
    "UK": "United Kingdom",
    "UAE": "United Arab Emirates",
    "Antigua & Barbuda": "Antigua and Barbuda",
    "Trinidad & Tobago": "Trinidad and Tobago",
    "St. Kitts & Nevis": "Saint Kitts and Nevis",
    "St. Kitts and Nevis": "Saint Kitts and Nevis",
    "St. Lucia": "Saint Lucia",
    "St. Vincent & Grenadines": "Saint Vincent and the Grenadines",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "St.Vincent & Grenadines": "Saint Vincent and the Grenadines",
    "Sao Tome & Principe": "Sao Tome and Principe",
    "São Tomé & Príncipe": "Sao Tome and Principe",
    "Hong Kong, China": "Hong Kong",
    "Hong Kong SAR, China": "Hong Kong",
    "Macao SAR, China": "Macao",
    "Palestine": "Palestine, State of",
    "West Bank and Gaza": "Palestine, State of",
    "Palestinian Adm. Areas": "Palestine, State of",
    
    # === MIP COW CODES ===
    "CUB": "Cuba",
    "IRQ": "Iraq",
    "KUW": "Kuwait",
    "LBR": "Liberia",
    "SAU": "Saudi Arabia",
    "CAN": "Canada",
    "IRN": "Iran",
    "SOM": "Somalia",
    "PER": "Peru",
    "HAI": "Haiti",
    "HON": "Honduras",
    "LEB": "Lebanon",
    "LIB": "Libya",
    "NIR": "Niger",
    "PHI": "Philippines",
    "SUD": "Sudan",
    "THI": "Thailand",
    "URU": "Uruguay",
    "YUG": "Yugoslavia",
    "BOS": "Bosnia and Herzegovina",
    "CAM": "Cameroon",
    "CAO": "Central African Republic",
    "CEN": "Central African Republic",
    "CDI": "Côte d'Ivoire",
    "GUA": "Guatemala",
}

def clean_country_name(name):
    """Clean country name before pycountry lookup"""
    if pd.isna(name):
        return None
    
    name = str(name).strip()
    
    # Strip "Government of" prefix (UCDP)
    if name.startswith("Government of "):
        name = name.replace("Government of ", "")
    
    # Apply manual mapping
    if name in manual_map:
        name = manual_map[name]
    
    return name


def get_iso3(country_name):
    """Get ISO3 code for a country name"""
    cleaned = clean_country_name(country_name)
    if cleaned is None:
        return None
    try:
        return pycountry.countries.lookup(cleaned).alpha_3
    except LookupError:
        return None

# Test again
failed_sample = ['Russia', 'Soviet Union', 'Korea, Rep.', 'Congo - Kinshasa', 'KUW', 'Turkiye', 'Bahamas, The']
for name in failed_sample:
    print(f"{name} -> {get_iso3(name)}")

Russia → RUS
Soviet Union → RUS
Korea, Rep. → KOR
Congo - Kinshasa → None
KUW → KWT
Turkiye → TUR
Bahamas, The → BHS


In [10]:
#congo
# Try different variations
test_congo = [
    "Democratic Republic of the Congo",
    "Congo, The Democratic Republic of the",
    "Congo",
]

for name in test_congo:
    try:
        result = pycountry.countries.lookup(name)
        print(f"{name} → {result.alpha_3}")
    except LookupError:
        print(f"{name} → NOT FOUND")

Democratic Republic of the Congo → NOT FOUND
Congo, The Democratic Republic of the → COD
Congo → COG


In [ ]:
# Update the Congo mappings in manual_map:
manual_map.update({
    "Congo - Kinshasa": "Congo, The Democratic Republic of the",
    "Congo, Dem. Rep.": "Congo, The Democratic Republic of the",
    "Congo, Democratic Republic of": "Congo, The Democratic Republic of the",
    "DR Congo": "Congo, The Democratic Republic of the",
    "DRC": "Congo, The Democratic Republic of the",
    "Democratic Republic of the Congo": "Congo, The Democratic Republic of the",
    "Congo - Brazzaville": "Congo",
    "Congo, Rep.": "Congo",
    "Congo, Republic of": "Congo",
    "Republic of Congo": "Congo",
})

# Test
print(f"Congo - Kinshasa -> {get_iso3('Congo - Kinshasa')}")
print(f"Congo - Brazzaville -> {get_iso3('Congo - Brazzaville')}")

Congo - Kinshasa → COD
Congo - Brazzaville → COG


In [12]:
# Re-run the check with updated mapping
failed = []
for name in sorted([n for n in all_country_names if pd.notna(n)]):
    if get_iso3(name) is None:
        failed.append(name)

print(f"Failed to match: {len(failed)}")

# Let's categorize what's left - show only likely countries (not orgs/regions/rebels)
print("\n=== Likely countries still failing ===")
for name in failed:
    # Skip obvious non-countries
    if any(x in name.lower() for x in ['regional', 'bank', 'fund', 'development', 'united nations', 
                                        'world', 'islamic state', 'government', 'rebels', 
                                        'income', 'oecd', 'euro', 'asia', 'africa', 'america',
                                        'caribbean', 'pacific', 'idb', 'ida', 'ibrd', '**',
                                        'multi-country', 'unspecified', ':', '*', 'elements']):
        continue
    # Skip acronyms (likely rebel groups)
    if name.isupper() and len(name) <= 6:
        continue
    print(f"  - {name}")

Failed to match: 524

=== Likely countries still failing ===
  - ALiR
  - ARS/UIC
  - Abdullah Azzam Brigades
  - Achik Special Dragon Party
  - Agwelek Forces
  - Al-Shabaab
  - Ansar Bayt al-Maqdis
  - Ansar Dine
  - Ansar al-Islam
  - Australia, United Kingdom, United States - Iraq
  - Benghazi Revolutionaries Shura Council
  - Bill & Melinda Gates Foundation
  - CPI-Maoist
  - Cameroon - Nigeria
  - Confederation of Mountain Peoples of the Caucasus
  - Czechoslovakia
  - Côte d’Ivoire
  - Djibouti - Eritrea
  - Dominica,  St. Martin, 
  - Early-demographic dividend
  - East Germany (GDR)
  - Eritrea - Ethiopia
  - FDP/SSAF
  - FLEC-FAC
  - Falkland Islands
  - Fragile and conflict affected situations
  - GUA, HON, COL
  - Global
  - Global Alliance for Vaccines & Immunization (GAVI)
  - Global Environment Facility (GEF)
  - Global Green Growth Institute (GGGI)
  - Global Partnership for Education
  - Groupe de Sécurité (Nyatura Kasongo)
  - Hamas
  - Heavily indebted poor countries

In [13]:
# Add these to manual_map:
manual_map.update({
    # Real countries still failing
    "Côte d'Ivoire": "Côte d'Ivoire",  # accent issue - try direct lookup
    "Russian Federation": "Russia",
    "Czechoslovakia": "Czechia",  # historical - map to successor
    "East Germany (GDR)": "Germany",  # historical
    "Yugoslavia": "Serbia",  # map to successor
    "Serbia and Montenegro": "Serbia",  # historical
    "Kosovo": "Kosovo",  # special case
    "Micronesia (Federated States of)": "Micronesia, Federated States of",
    "Micronesia, Fed. Sts.": "Micronesia, Federated States of",
    "Somalia, Fed. Rep.": "Somalia",
    "Netherlands Antilles": "Netherlands",  # historical
    "Israel and the Occupied Palestinian Territory": "Israel",
    "Falkland Islands": "Falkland Islands (Malvinas)",
    "Reunion": "Réunion",
    "Northern Marianas": "Northern Mariana Islands",
    "Virgin Islands (UK)": "Virgin Islands, British",
    "Wallis & Futuna": "Wallis and Futuna",
    "St. Helena": "Saint Helena, Ascension and Tristan da Cunha",
    "Northern Cyprus": "Cyprus",  # disputed - map to Cyprus
})

# Test the tricky ones
test_names = ["Côte d'Ivoire", "Russian Federation", "Micronesia, Fed. Sts.", "Kosovo"]
for name in test_names:
    print(f"{name} -> {get_iso3(name)}")

Côte d'Ivoire -> CIV
Russian Federation -> None
Micronesia, Fed. Sts. -> FSM
Kosovo -> None


In [14]:
# Debug Russian Federation
test_russia = ["Russia", "Russian Federation", "RUS"]
for name in test_russia:
    try:
        result = pycountry.countries.lookup(name)
        print(f"{name} -> {result.alpha_3}")
    except LookupError:
        print(f"{name} -> NOT FOUND")

Russia -> NOT FOUND
Russian Federation -> RUS
RUS -> RUS


In [15]:
# Fix the Russia mapping - remove the wrong one, keep correct direction
manual_map.update({
    "Russia": "Russian Federation",  # pycountry wants "Russian Federation"
    "Soviet Union": "Russian Federation",
})

# Remove the incorrect reverse mapping if it exists
if "Russian Federation" in manual_map and manual_map["Russian Federation"] == "Russia":
    del manual_map["Russian Federation"]

# Test
test_names = ["Russia", "Russian Federation", "Soviet Union"]
for name in test_names:
    print(f"{name} → {get_iso3(name)}")

Russia → RUS
Russian Federation → RUS
Soviet Union → RUS


In [16]:
# Special cases without official ISO codes
special_codes = {
    "Kosovo": "XKX",
}

def get_iso3(country_name):
    """Get ISO3 code for a country name"""
    cleaned = clean_country_name(country_name)
    if cleaned is None:
        return None
    
    # Check special cases first
    if cleaned in special_codes:
        return special_codes[cleaned]
    
    try:
        return pycountry.countries.lookup(cleaned).alpha_3
    except LookupError:
        return None

# Test
print(f"Kosovo -> {get_iso3('Kosovo')}")

Kosovo -> XKX


In [18]:
# Final check
failed = []
for name in sorted([n for n in all_country_names if pd.notna(n)]):
    if get_iso3(name) is None:
        failed.append(name)

print(f"Total still failing: {len(failed)}")

Total still failing: 505


In [19]:
# Check how many rows in each dataset will have valid ISO3 codes

print("=== Coverage check ===\n")

# CPJ
cpj['iso3'] = cpj['Country'].apply(get_iso3)
matched = cpj['iso3'].notna().sum()
total = len(cpj)
print(f"CPJ: {matched}/{total} rows matched ({100*matched/total:.1f}%)")

# SIPRI - check recipients (your target countries)
sipri['recipient_iso3'] = sipri['recipient'].apply(get_iso3)
matched = sipri['recipient_iso3'].notna().sum()
total = len(sipri)
print(f"SIPRI recipients: {matched}/{total} rows matched ({100*matched/total:.1f}%)")

# MIP
mip['iso3'] = mip['target_country'].apply(get_iso3)
matched = mip['iso3'].notna().sum()
total = len(mip)
print(f"MIP: {matched}/{total} rows matched ({100*matched/total:.1f}%)")

# COLDAT
coldat['colony_iso3'] = coldat['colony'].apply(get_iso3)
matched = coldat['colony_iso3'].notna().sum()
total = len(coldat)
print(f"COLDAT colonies: {matched}/{total} rows matched ({100*matched/total:.1f}%)")

# AidData - check recipients
aiddata['recipient_iso3'] = aiddata['recipient'].apply(get_iso3)
matched = aiddata['recipient_iso3'].notna().sum()
total = len(aiddata)
print(f"AidData recipients: {matched}/{total} rows matched ({100*matched/total:.1f}%)")

# Debt
debt['debtor_iso3'] = debt['debtor'].apply(get_iso3)
matched = debt['debtor_iso3'].notna().sum()
total = len(debt)
print(f"Debt debtors: {matched}/{total} rows matched ({100*matched/total:.1f}%)")

# ODA
oda['iso3'] = oda['country'].apply(get_iso3)
matched = oda['iso3'].notna().sum()
total = len(oda)
print(f"ODA: {matched}/{total} rows matched ({100*matched/total:.1f}%)")

=== Coverage check ===

CPJ: 763/763 rows matched (100.0%)
SIPRI recipients: 12953/13169 rows matched (98.4%)
MIP: 130/146 rows matched (89.0%)
COLDAT colonies: 160/161 rows matched (99.4%)
AidData recipients: 75626/82112 rows matched (92.1%)
Debt debtors: 19980/24760 rows matched (80.7%)
ODA: 5122/6679 rows matched (76.7%)


In [20]:
# Check MIP failures
print("=== MIP unmatched ===")
mip_failed = mip[mip['iso3'].isna()]['target_country'].unique()
print(mip_failed)

=== MIP unmatched ===
['SIE' 'YUG' 'KEN, TAZ' 'ETM' 'INS' 'IRQ, SYR'
 'AFG, IRQ, SYR, TUR, SOM, YEM, DJI, LBY, CUB, Central Africa, EGY, JOR'
 'Dominica,  St. Martin, ' 'GUA, HON, COL' 'ICE']


In [21]:
# Check ODA failures - sample
print("\n=== ODA unmatched (sample) ===")
oda_failed = oda[oda['iso3'].isna()]['country'].unique()
print(oda_failed[:20])


=== ODA unmatched (sample) ===
['Africa Eastern and Southern' 'Africa Western and Central' 'Arab World'
 'Central Europe and the Baltics' 'Caribbean small states'
 'East Asia & Pacific (excluding high income)'
 'Early-demographic dividend' 'East Asia & Pacific'
 'Europe & Central Asia (excluding high income)' 'Europe & Central Asia'
 'Euro area' 'European Union' 'Fragile and conflict affected situations'
 'High income' 'Heavily indebted poor countries (HIPC)' 'IBRD only'
 'IDA & IBRD total' 'IDA total' 'IDA blend' 'IDA only']


In [22]:
# Add missing MIP COW codes
manual_map.update({
    "SIE": "Sierra Leone",
    "YUG": "Serbia",  # Yugoslavia successor
    "ETM": "East Timor",
    "INS": "Indonesia",
    "ICE": "Iceland",
})

# These are multi-country interventions - we'll handle separately or skip:
# 'KEN, TAZ' - Kenya, Tanzania
# 'IRQ, SYR' - Iraq, Syria
# 'AFG, IRQ, SYR, TUR, SOM, YEM, DJI, LBY, CUB, Central Africa, EGY, JOR'
# 'Dominica, St. Martin,'
# 'GUA, HON, COL'

# Re-check MIP
mip['iso3'] = mip['target_country'].apply(get_iso3)
matched = mip['iso3'].notna().sum()
print(f"MIP: {matched}/{len(mip)} rows matched ({100*matched/len(mip):.1f}%)")

# Show what's still missing
print("\nStill missing:")
print(mip[mip['iso3'].isna()]['target_country'].unique())

MIP: 140/146 rows matched (95.9%)

Still missing:
['KEN, TAZ' 'ETM' 'IRQ, SYR'
 'AFG, IRQ, SYR, TUR, SOM, YEM, DJI, LBY, CUB, Central Africa, EGY, JOR'
 'Dominica,  St. Martin, ' 'GUA, HON, COL']


In [23]:
# Debug East Timor
test_timor = ["East Timor", "Timor-Leste", "TLS"]
for name in test_timor:
    try:
        result = pycountry.countries.lookup(name)
        print(f"{name} → {result.alpha_3}")
    except LookupError:
        print(f"{name} → NOT FOUND")

East Timor → NOT FOUND
Timor-Leste → TLS
TLS → TLS


In [24]:
# Fix East Timor mapping (after checking what works above)
manual_map.update({
    "ETM": "Timor-Leste",  # adjust based on test results
})

# COW code mappings for multi-country splits
cow_to_name = {
    "KEN": "Kenya",
    "TAZ": "Tanzania",
    "IRQ": "Iraq", 
    "SYR": "Syria",
    "AFG": "Afghanistan",
    "TUR": "Turkey",
    "SOM": "Somalia",
    "YEM": "Yemen",
    "DJI": "Djibouti",
    "LBY": "Libya",
    "CUB": "Cuba",
    "EGY": "Egypt",
    "JOR": "Jordan",
    "GUA": "Guatemala",
    "HON": "Honduras",
    "COL": "Colombia",
}

def split_multi_country(df, country_col):
    """Split rows with multiple countries into separate rows"""
    new_rows = []
    
    for idx, row in df.iterrows():
        target = row[country_col]
        
        # Check if it's a multi-country entry (contains comma)
        if pd.notna(target) and ',' in str(target):
            # Split by comma and clean
            countries = [c.strip() for c in str(target).split(',')]
            
            for country in countries:
                # Skip empty or problematic entries
                if not country or country == 'Central Africa' or country == 'St. Martin':
                    continue
                # Map COW code to name if needed
                if country in cow_to_name:
                    country = cow_to_name[country]
                
                new_row = row.copy()
                new_row[country_col] = country
                new_rows.append(new_row)
        else:
            new_rows.append(row)
    
    return pd.DataFrame(new_rows)

# Apply to MIP
mip_expanded = split_multi_country(mip, 'target_country')
print(f"MIP rows: {len(mip)} → {len(mip_expanded)} (after splitting)")

# Re-apply ISO3
mip_expanded['iso3'] = mip_expanded['target_country'].apply(get_iso3)
matched = mip_expanded['iso3'].notna().sum()
print(f"MIP: {matched}/{len(mip_expanded)} rows matched ({100*matched/len(mip_expanded):.1f}%)")

# Check what's still missing
still_missing = mip_expanded[mip_expanded['iso3'].isna()]['target_country'].unique()
print(f"\nStill missing: {still_missing}")

MIP rows: 146 → 160 (after splitting)
MIP: 160/160 rows matched (100.0%)

Still missing: []


In [25]:
# Fix East Timor mapping
manual_map.update({
    "ETM": "Timor-Leste",
    "East Timor": "Timor-Leste",
})

In [26]:
# Final coverage check
print("=== FINAL COVERAGE ===\n")

# CPJ
cpj['iso3'] = cpj['Country'].apply(get_iso3)
print(f"CPJ: {cpj['iso3'].notna().sum()}/{len(cpj)} ({100*cpj['iso3'].notna().mean():.1f}%)")

# SIPRI
sipri['recipient_iso3'] = sipri['recipient'].apply(get_iso3)
sipri['supplier_iso3'] = sipri['supplier'].apply(get_iso3)
print(f"SIPRI recipients: {sipri['recipient_iso3'].notna().sum()}/{len(sipri)} ({100*sipri['recipient_iso3'].notna().mean():.1f}%)")

# MIP (expanded)
print(f"MIP: {mip_expanded['iso3'].notna().sum()}/{len(mip_expanded)} ({100*mip_expanded['iso3'].notna().mean():.1f}%)")

# UCDP - needs work on target_location (has "Country: Region" format)
ucdp['target_country'] = ucdp['target_location'].str.split(':').str[0].str.strip()
ucdp['target_iso3'] = ucdp['target_country'].apply(get_iso3)
print(f"UCDP targets: {ucdp['target_iso3'].notna().sum()}/{len(ucdp)} ({100*ucdp['target_iso3'].notna().mean():.1f}%)")

# COLDAT
coldat['colony_iso3'] = coldat['colony'].apply(get_iso3)
print(f"COLDAT: {coldat['colony_iso3'].notna().sum()}/{len(coldat)} ({100*coldat['colony_iso3'].notna().mean():.1f}%)")

# AidData
aiddata['recipient_iso3'] = aiddata['recipient'].apply(get_iso3)
print(f"AidData: {aiddata['recipient_iso3'].notna().sum()}/{len(aiddata)} ({100*aiddata['recipient_iso3'].notna().mean():.1f}%)")

# Debt
debt['debtor_iso3'] = debt['debtor'].apply(get_iso3)
print(f"Debt: {debt['debtor_iso3'].notna().sum()}/{len(debt)} ({100*debt['debtor_iso3'].notna().mean():.1f}%)")

# ODA
oda['iso3'] = oda['country'].apply(get_iso3)
print(f"ODA: {oda['iso3'].notna().sum()}/{len(oda)} ({100*oda['iso3'].notna().mean():.1f}%)")

=== FINAL COVERAGE ===

CPJ: 763/763 (100.0%)
SIPRI recipients: 12953/13169 (98.4%)
MIP: 160/160 (100.0%)
UCDP targets: 6484/6899 (94.0%)
COLDAT: 160/161 (99.4%)
AidData: 75626/82112 (92.1%)
Debt: 19980/24760 (80.7%)
ODA: 5122/6679 (76.7%)


In [27]:
# Filter to only rows with valid ISO3 codes and save

# CPJ (target)
cpj_clean = cpj[cpj['iso3'].notna()].copy()
cpj_clean.to_csv(PROCESSED / "target" / "target_journalist_killings.csv", index=False)
print(f"Saved CPJ: {len(cpj_clean)} rows")

# SIPRI
sipri_clean = sipri[sipri['recipient_iso3'].notna()].copy()
sipri_clean.to_csv(PROCESSED / "military" / "sipri_trade_register.csv", index=False)
print(f"Saved SIPRI: {len(sipri_clean)} rows")

# MIP (use expanded version)
mip_clean = mip_expanded[mip_expanded['iso3'].notna()].copy()
mip_clean.to_csv(PROCESSED / "military" / "mip_us_interventions.csv", index=False)
print(f"Saved MIP: {len(mip_clean)} rows")

# UCDP
ucdp_clean = ucdp[ucdp['target_iso3'].notna()].copy()
ucdp_clean.to_csv(PROCESSED / "military" / "ucdp_esd_ty.csv", index=False)
print(f"Saved UCDP: {len(ucdp_clean)} rows")

# COLDAT
coldat_clean = coldat[coldat['colony_iso3'].notna()].copy()
coldat_clean.to_csv(PROCESSED / "colonial" / "coldat_colonial_ties.csv", index=False)
print(f"Saved COLDAT: {len(coldat_clean)} rows")

# AidData
aiddata_clean = aiddata[aiddata['recipient_iso3'].notna()].copy()
aiddata_clean.to_csv(PROCESSED / "economic" / "aiddata_bilateral_flows.csv", index=False)
print(f"Saved AidData: {len(aiddata_clean)} rows")

# Debt
debt_clean = debt[debt['debtor_iso3'].notna()].copy()
debt_clean.to_csv(PROCESSED / "economic" / "worldbank_bilateral_debt.csv", index=False)
print(f"Saved Debt: {len(debt_clean)} rows")

# ODA
oda_clean = oda[oda['iso3'].notna()].copy()
oda_clean.to_csv(PROCESSED / "control-variable" / "oda_received_by_country.csv", index=False)
print(f"Saved ODA: {len(oda_clean)} rows")

print("\n YAY All datasets standardized and saved!")

Saved CPJ: 763 rows
Saved SIPRI: 12953 rows
Saved MIP: 160 rows
Saved UCDP: 6484 rows
Saved COLDAT: 160 rows
Saved AidData: 75626 rows
Saved Debt: 19980 rows
Saved ODA: 5122 rows

 YAY All datasets standardized and saved!
